---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevincrotty.rice.edu/)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
start = 1970
end = 2022

# GET INFLATION DATA

In [2]:
from pandas_datareader import DataReader as pdr

cpi = pdr('CPIAUCSL', 'fred', start=f"{start-1}-12-01", end=f"{end+1}-01-01")
cpi = cpi.resample('Y').last()

inflation = cpi.pct_change().iloc[1:-1]
inflation.columns = ["inflation"]
inflation.index = inflation.index.map(lambda x: x.year)
inflation.index.name = "year"
inflation.tail()

,inflation
year,
2018,0.020024
2019,0.023140
2020,0.013220
2021,0.071945
2022,0.064449


# GET STOCK MARKET RETURNS

In [3]:
ff = pdr('F-F_Research_Data_Factors', 'famafrench', start=start, end=end)[1]/100
market = ff['Mkt-RF'] + ff['RF']
market.name = "market"
market.index = market.index.astype(str).astype(int)
market.tail()

Date
2018   -0.0512
2019    0.3043
2020    0.2411
2021    0.2360
2022   -0.2017
Name: market, dtype: float64

# MERGE DATA

In [4]:
df = inflation.join(market)
df["real"] = (1+df.market) / (1+df.inflation) - 1
df.mean()

inflation    0.040252
market       0.119057
real         0.077097
dtype: float64

# FIGURE 1

In [5]:
import plotly.graph_objects as go

trace = go.Scatter(
    x=df.index.to_list(),
    y=df.inflation,
    mode="lines",
    hovertemplate="%{x}<br>inflation = %{y:.2%}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Inflation Rate",
    yaxis_tickformat=".0%",
    template="plotly_white"
)
fig.show()

# FIGURE 2

In [6]:
df = (1+df).cumprod().dropna()
trace1 = go.Scatter(
    x=df.index.to_list(),
    y=df.market,
    mode="lines",
    name="nominal $",
    hovertemplate="Accumulation in Nominal $ = %{y:0.2f}<extra></extra>"
)

trace2 = go.Scatter(
    x=df.index.to_list(),
    y=df.real,
    mode="lines",
    name="constant $",
    hovertemplate="Accumulation in Constant $ = %{y:0.2f}<extra></extra>"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Compound Return",
    yaxis_tickformat="$.0f",
    hovermode="x unified",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    template="plotly_white",
)
fig.show()